In [1]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr

In [2]:
with open("simu6b_data/simu6b_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
df_study = df_study.loc[df_study.session < 100]
df_test = df_test.loc[df_test.session < 100]

In [3]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,list,session
0,120,459,BELT,DIAPER,0,0
1,1315,1260,SKETCH,SEAFOOD,0,0
2,357,723,COMPANION,HOSTESS,0,0
3,1551,1202,VEAL,ROCKET,0,0
4,1081,1066,PLANE,PIGMENT,0,0
...,...,...,...,...,...,...
7195,1019,1143,PASSAGE,PUPIL,5,99
7196,293,1375,CHERRY,STEAM,5,99
7197,871,927,MARKET,MOVIE,5,99
7198,1392,1486,STREAM,TORTOISE,5,99


In [4]:
df_test

,test_itemno1,test_itemno2,test_item,correct_ans,order,rep,test,list,session,test_pos,pair_idx
0,51,-1,ASTRONAUT,1083,2,1,1,0,0,1,8
1,1202,-1,ROCKET,1551,2,1,1,0,0,2,4
2,979,-1,OPERATOR,142,1,1,1,0,0,3,11
3,134,-1,BLADE,105,1,1,1,0,0,4,9
4,1081,-1,PLANE,1066,1,1,1,0,0,5,5
...,...,...,...,...,...,...,...,...,...,...,...
14395,1019,-1,PASSAGE,1143,1,1,2,5,99,20,7196
14396,487,-1,DOORBELL,788,2,1,2,5,99,21,7189
14397,723,-1,HOSTESS,1378,2,1,2,5,99,22,7192
14398,196,-1,BULLY,786,1,1,2,5,99,23,7190


### Run CMR-IA

In [5]:
s_mat = np.load('../../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [6]:
params = cmr.make_default_params()
params.update(learn_while_retrieving=True, nitems_in_accumulator = 96)
# params.update(
#     beta_enc = 0.5,
#     beta_rec = 0.1, # beta for retrieved item
#     beta_cue = 0.45, # beta for cue
#     beta_rec_post = 0.99,
#     beta_distract = 0.1,
#     gamma_fc = 0.35,
#     gamma_cf = 0.35,
#     s_fc = 0.1,
#     s_cf = 0.1,
#     c_thresh = 0.4,
#     c_thresh_ass = 0.01,
#     kappa = 0.02,
#     lamb = 0.01,
#     eta = 0.005,
#     phi_s = 1,
#     phi_d = 0.6,
#     nitems_in_accumulator = 96,
#     d_ass = 0.7,
#     learn_while_retrieving = True,
#     omega = 5,
#     alpha = 1
# )
# params

In [7]:
# pso results
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from optimization_utils import make_boundary

_,_,what_to_fit = make_boundary(sim_name='6b')
optim_params = np.loadtxt("../../Modeling/CMR_IA/fitting/outparams_backup/6b_230809_100-100.txt")
for pname, pvalue in zip(what_to_fit, optim_params):
    params[pname] = pvalue
params

{'beta_enc': 0.681424,
 'beta_rec': 0.380198,
 'beta_cue': 0.951729,
 'beta_rec_post': 0.62647,
 'beta_distract': 0.150624,
 'phi_s': 6.374683,
 'phi_d': 0.679841,
 's_cf': 0.692925,
 's_fc': 0.028039,
 'kappa': 0.440224,
 'eta': 0.109809,
 'omega': 1.703231,
 'alpha': 0.59174,
 'c_thresh': 0.11128,
 'c_thresh_itm': 0.5,
 'c_thresh_ass': 0.5,
 'd_ass': 0.685889,
 'lamb': 0.09386,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 96,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'thresh_sigma': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.342828,
 'gamma_cf': 0.52738,
 'use_new_context': False}

In [8]:
df_simu, f_in, f_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat, mode='CR-CR')
# df_simu['test_pos'] = np.tile(np.arange(1,25),6000)
df_simu['test_pos'] = np.tile(np.arange(1,25),600)
df_simu = df_simu.merge(df_test,on=['session','list', 'test_itemno1','test_itemno2', 'test_pos'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

CMR2 Time: 38.09381175041199


,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct
0,0,0,51,-1,-1.0,-1.0,-1.000000,1,ASTRONAUT,1083,2,1,1,8,False
1,0,0,1202,-1,1551.0,2750.0,0.698624,2,ROCKET,1551,2,1,1,4,True
2,0,0,979,-1,-1.0,-1.0,-1.000000,3,OPERATOR,142,1,1,1,11,False
3,0,0,134,-1,-1.0,-1.0,-1.000000,4,BLADE,105,1,1,1,9,False
4,0,0,1081,-1,-1.0,-1.0,-1.000000,5,PLANE,1066,1,1,1,5,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14395,99,5,1019,-1,786.0,3600.0,0.144349,20,PASSAGE,1143,1,1,2,7196,False
14396,99,5,487,-1,788.0,280.0,0.965276,21,DOORBELL,788,2,1,2,7189,True
14397,99,5,723,-1,665.0,590.0,0.565380,22,HOSTESS,1378,2,1,2,7192,False
14398,99,5,196,-1,786.0,760.0,0.921066,23,BULLY,786,1,1,2,7190,True


In [9]:
# get condition and congruence
df_cond = df_simu.groupby(["pair_idx","test"])['order'].mean().to_frame(name='corr_rate').reset_index()
df_cond = df_cond.pivot_table(index='pair_idx',columns='test',values='corr_rate').reset_index()
df_cond.columns = ['pair_idx','test1','test2']

def cond(x):
    test1 = x['test1']
    test2 = x['test2']
    if test1 == 1 and test2 == 1:
        return 'F-F'
    elif test1 == 1 and test2 == 2:
        return 'F-B'
    elif test1 == 2 and test2 == 1:
        return 'B-F'
    elif test1 == 2 and test2 == 2:
        return 'B-B'

df_cond['cond'] = df_cond.apply(lambda x:cond(x),axis=1)
df_cond['cong'] = df_cond.apply(lambda x: 'Identical' if x['cond'] == 'F-F' or x['cond'] == 'B-B' else 'Reversed',axis=1)
df_cond

,pair_idx,test1,test2,cond,cong
0,1,1.0,1.0,F-F,Identical
1,2,2.0,1.0,B-F,Reversed
2,3,1.0,1.0,F-F,Identical
3,4,2.0,1.0,B-F,Reversed
4,5,1.0,2.0,F-B,Reversed
...,...,...,...,...,...
7195,7196,2.0,1.0,B-F,Reversed
7196,7197,1.0,1.0,F-F,Identical
7197,7198,2.0,1.0,B-F,Reversed
7198,7199,1.0,2.0,F-B,Reversed


In [10]:
# merge condition and congruence
pairidx2cond = df_cond.loc[:,['pair_idx','cond']].set_index("pair_idx").to_dict()['cond']
pairidx2cong = df_cond.loc[:,['pair_idx','cong']].set_index("pair_idx").to_dict()['cong']
df_simu['cond'] = df_simu.apply(lambda x:pairidx2cond[x['pair_idx']],axis=1)
df_simu['cong'] = df_simu.apply(lambda x:pairidx2cong[x['pair_idx']],axis=1)
df_simu.head(24)

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong
0,0,0,51,-1,-1.0,-1.0,-1.000000,1,ASTRONAUT,1083,2,1,1,8,False,B-B,Identical
1,0,0,1202,-1,1551.0,2750.0,0.698624,2,ROCKET,1551,2,1,1,4,True,B-F,Reversed
2,0,0,979,-1,-1.0,-1.0,-1.000000,3,OPERATOR,142,1,1,1,11,False,F-F,Identical
3,0,0,134,-1,-1.0,-1.0,-1.000000,4,BLADE,105,1,1,1,9,False,F-B,Reversed
4,0,0,1081,-1,-1.0,-1.0,-1.000000,5,PLANE,1066,1,1,1,5,False,F-B,Reversed
5,0,0,357,-1,723.0,610.0,0.703930,6,COMPANION,723,1,1,1,3,True,F-F,Identical
6,0,0,831,-1,1435.0,1980.0,0.708035,7,LITERATURE,1435,2,1,1,7,True,B-B,Identical
7,0,0,1188,-1,-1.0,-1.0,-1.000000,8,RIB,641,2,1,1,12,False,B-F,Reversed
8,0,0,1260,-1,1315.0,680.0,0.658526,9,SEAFOOD,1315,2,1,1,2,True,B-F,Reversed
9,0,0,12,-1,-1.0,-1.0,-1.000000,10,AIRPLANE,226,1,1,1,10,False,F-B,Reversed


In [11]:
# get f_in
sessions = np.unique(df_simu.session.to_numpy())
tmp_corr_fin = []
tmp_omean_fin = []
tmp_omax_fin = []

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    tmp = np.concatenate((tmp1, tmp2))
    tmp = np.sort(tmp)

    df_tmp1 = df_simu.loc[df_simu.session == sess]
    tmp_test = df_tmp1.test_itemno1
    tmp_corr = df_tmp1.correct_ans
    # tmp_test = df_simu.loc[df_simu.session == sess, 'test_itemno']
    # tmp_corr = df_simu.loc[df_simu.session == sess, 'correct_ans']
    testid = np.searchsorted(tmp, tmp_test)
    corrid = np.searchsorted(tmp, tmp_corr)

    fin_sess = f_dif[sess]
    # tmp_corr_fin = []
    # tmp_omean_fin = []
    # tmp_omax_fin = []
    for i in range(len(testid)):
        tmp_corr_fin.append(fin_sess[i][corrid[i]])
        tmp_omean_fin.append(np.mean(np.delete(fin_sess[i], corrid[i])))
        tmp_omax_fin.append(np.delete(fin_sess[i], corrid[i]).max())
    # df_simu.loc[df_simu.session == sess, 'corr_fin']= tmp_corr_fin
    # df_simu.loc[df_simu.session == sess, 'omean_fin'] = tmp_omean_fin
    # df_simu.loc[df_simu.session == sess, 'omax_fin'] = tmp_omax_fin

df_simu['corr_fin']= tmp_corr_fin
df_simu['omean_fin'] = tmp_omean_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,list,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,0,51,-1,-1.0,-1.0,-1.000000,1,ASTRONAUT,1083,2,1,1,8,False,B-B,Identical,-0.183877,-0.837549,-0.338363
1,0,0,1202,-1,1551.0,2750.0,0.698624,2,ROCKET,1551,2,1,1,4,True,B-F,Reversed,0.179725,-0.848148,-0.190495
2,0,0,979,-1,-1.0,-1.0,-1.000000,3,OPERATOR,142,1,1,1,11,False,F-F,Identical,-0.266787,-0.870753,-0.400814
3,0,0,134,-1,-1.0,-1.0,-1.000000,4,BLADE,105,1,1,1,9,False,F-B,Reversed,-0.132593,-0.861277,-0.298594
4,0,0,1081,-1,-1.0,-1.0,-1.000000,5,PLANE,1066,1,1,1,5,False,F-B,Reversed,-0.025643,-0.865560,-0.339170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14395,99,5,1019,-1,786.0,3600.0,0.144349,20,PASSAGE,1143,1,1,2,7196,False,B-F,Reversed,-0.186727,-0.786962,-0.373456
14396,99,5,487,-1,788.0,280.0,0.965276,21,DOORBELL,788,2,1,2,7189,True,B-B,Identical,3.206532,-0.704418,1.418255
14397,99,5,723,-1,665.0,590.0,0.565380,22,HOSTESS,1378,2,1,2,7192,False,F-B,Reversed,0.963497,-0.667192,1.029669
14398,99,5,196,-1,786.0,760.0,0.921066,23,BULLY,786,1,1,2,7190,True,F-F,Identical,1.837592,-0.697257,0.909715


In [12]:
df_fin = df_simu.groupby(['cong','test'])[['corr_fin','omean_fin','omax_fin']].mean().reset_index()
df_fin['dif'] = df_fin['corr_fin'] - df_fin['omax_fin']
df_fin

,cong,test,corr_fin,omean_fin,omax_fin,dif
0,Identical,1,0.285616,-0.797839,0.023393,0.262223
1,Identical,2,0.542523,-0.759281,0.617043,-0.074520
2,Reversed,1,0.289630,-0.797665,0.013385,0.276245
3,Reversed,2,0.593983,-0.770208,0.405713,0.188270


### Analysis

In [13]:
# anal_perform used by pso
import sys
sys.path.append("../../Modeling/CMR_IA/fitting")
from object_funcs import anal_perform_6b as anal_perform

In [14]:
# def anal_perform(df_simu):

#     # get pair
#     df_pair = pd.pivot_table(df_simu,index='pair_idx',columns='test', values= 'correct')
#     df_pair.columns = ['test1','test2']
#     test2_rsp = pd.Categorical(df_pair.test2, categories=[1,0])
#     test1_rsp = pd.Categorical(df_pair.test1, categories=[1,0])
#     df_tab = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize=False, dropna=False)
#     df_tab_norm = pd.crosstab(index=test2_rsp,columns=test1_rsp, rownames=['test2'], colnames=['test1'], normalize='all', dropna=False)
#     t1_t2 = df_tab_norm[1][1] # 1, 2
#     t1_f2 = df_tab_norm[1][0]
#     f1_t2 = df_tab_norm[0][1]
#     f1_f2 = df_tab_norm[0][0]
#     # print(df_tab)
#     # print(df_tab_norm)
#     # print(t1_t2, t1_f2, f1_t2, f1_f2)

#     # compute" Q
#     def Yule_Q(A, B, C, D):
#         return (A * D - B * C) / (A * D + B * C)
#     q = Yule_Q(df_tab[1][1]+0.5,df_tab[0][1]+0.5,df_tab[1][0]+0.5,df_tab[0][0]+0.5)  # add 0.5
#     # print("Q: ", q)

#     return t1_t2, t1_f2, f1_t2, f1_f2, q

In [15]:
subjects = np.unique(df_simu.session)
inde_stats = []
reve_stats = []
for subj in subjects:
    df_subj_inde = df_simu.query(f"session == {subj} and cong == 'Identical'").copy()
    inde_stats.append(list(anal_perform(df_subj_inde)))

    df_subj_reve = df_simu.query(f"session == {subj} and cong == 'Reversed'").copy()
    reve_stats.append(list(anal_perform(df_subj_reve)))

In [16]:
np.array(inde_stats)

array([[0.33333333, 0.05555556, 0.02777778, 0.58333333, 0.97247706],
       [0.33333333, 0.02777778, 0.02777778, 0.61111111, 0.98412698],
       [0.47222222, 0.02777778, 0.02777778, 0.47222222, 0.98541329],
       [0.36111111, 0.        , 0.13888889, 0.5       , 0.97821782],
       [0.16666667, 0.02777778, 0.05555556, 0.75      , 0.95890411],
       [0.30555556, 0.        , 0.02777778, 0.66666667, 0.99469027],
       [0.30555556, 0.        , 0.08333333, 0.61111111, 0.9865643 ],
       [0.30555556, 0.02777778, 0.05555556, 0.61111111, 0.97142857],
       [0.27777778, 0.02777778, 0.08333333, 0.61111111, 0.95652174],
       [0.33333333, 0.        , 0.11111111, 0.55555556, 0.98259188],
       [0.33333333, 0.05555556, 0.02777778, 0.58333333, 0.97247706],
       [0.47222222, 0.        , 0.05555556, 0.47222222, 0.99186992],
       [0.25      , 0.        , 0.08333333, 0.66666667, 0.98507463],
       [0.19444444, 0.02777778, 0.02777778, 0.75      , 0.97841727],
       [0.5       , 0.        , 0.

In [17]:
np.array(reve_stats)

array([[0.36111111, 0.        , 0.13888889, 0.5       , 0.97821782],
       [0.38888889, 0.02777778, 0.05555556, 0.52777778, 0.97382199],
       [0.25      , 0.05555556, 0.08333333, 0.61111111, 0.92134831],
       [0.33333333, 0.02777778, 0.05555556, 0.58333333, 0.97247706],
       [0.13888889, 0.02777778, 0.02777778, 0.80555556, 0.97264438],
       [0.25      , 0.        , 0.13888889, 0.61111111, 0.97459584],
       [0.41666667, 0.05555556, 0.13888889, 0.38888889, 0.88469602],
       [0.33333333, 0.02777778, 0.08333333, 0.55555556, 0.95984704],
       [0.30555556, 0.08333333, 0.        , 0.61111111, 0.9865643 ],
       [0.25      , 0.05555556, 0.11111111, 0.58333333, 0.89559165],
       [0.30555556, 0.        , 0.08333333, 0.61111111, 0.9865643 ],
       [0.22222222, 0.08333333, 0.05555556, 0.63888889, 0.91606715],
       [0.38888889, 0.02777778, 0.11111111, 0.47222222, 0.94817658],
       [0.16666667, 0.08333333, 0.        , 0.75      , 0.98060942],
       [0.27777778, 0.02777778, 0.

In [18]:
np.mean(inde_stats,axis=0)

array([0.3125    , 0.03166667, 0.04333333, 0.6125    , 0.97240297])

In [19]:
np.mean(reve_stats,axis=0)

array([0.32472222, 0.03166667, 0.0875    , 0.55611111, 0.95162105])

In [20]:
inde_stats_mean = np.mean(inde_stats,axis=0)
reve_stats_mean = np.mean(reve_stats,axis=0)
inde_ground_truth = np.array([0.319, 0.006, 0.012, 0.663, 0.94])
reve_ground_truth = np.array([0.293, 0.049, 0.122, 0.537, 0.96])
err = (np.mean(np.power(inde_stats_mean-inde_ground_truth,2)) + np.mean(np.power(reve_stats_mean-reve_ground_truth,2)))/2
err

0.0008215443196295067

### Whole

In [21]:
dfgp_test_order = df_simu.groupby(['rep','test','order']).correct.mean().to_frame(name='corr_rate').reset_index()
dfgp_test_order

,rep,test,order,corr_rate
0,1,1,1,0.346944
1,1,1,2,0.353611
2,1,2,1,0.389167
3,1,2,2,0.378889


In [22]:
df_simu.groupby(['cong','test']).correct.mean()

cong       test
Identical  1       0.344167
           2       0.355833
Reversed   1       0.356389
           2       0.412222
Name: correct, dtype: float64

In [23]:
def contingency_tab(x):
    tmp_x = pd.pivot_table(x,index='pair_idx',columns='test', values= 'correct')
    test1 = pd.Categorical(tmp_x[1], categories=[1, 0])
    test2 = pd.Categorical(tmp_x[2], categories=[1, 0])
    return pd.crosstab(index=test2,columns=test1, normalize='all')

In [24]:
dfgp_rep_cong = df_simu.groupby(['cong']).apply(contingency_tab)
dfgp_rep_cong

col_0                   1         0
cong      row_0                    
Identical 1      0.312500  0.043333
          0      0.031667  0.612500
Reversed  1      0.324722  0.087500
          0      0.031667  0.556111

In [25]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)

In [26]:
tab_identical = dfgp_rep_cong.loc['Identical']
Yule_Q(tab_identical[1][1],tab_identical[0][1],tab_identical[1][0],tab_identical[0][0])

0.9857637400441641

In [27]:
tab_reversed = dfgp_rep_cong.loc['Reversed']
Yule_Q(tab_reversed[1][1],tab_reversed[0][1],tab_reversed[1][0],tab_reversed[0][0])

0.9697758819786487

In [28]:
Yule_Q(0.319, 0.012, 0.006, 0.663)

0.9993193709853524

In [29]:
Yule_Q(0.293, 0.122, 0.049, 0.537)

0.9267935757627709